In [8]:
import os
import pathlib
import subprocess
import ray
from typing import Tuple

@ray.remote(num_cpus=16)
def call_ampl(instance_name:str, time_limit:int)->Tuple[str, str]:
    template: str
    template_filename: str
    model_filename: str
    model_filename = "CVRPPT14042025.mod"
    template_filename = "CVRPPT_run_template"
    workdir = pathlib.Path("~/gemilang/cvrpptpl-python").expanduser()

    output_dir = workdir/"ampl_outputs"
    output_dir.mkdir(parents=True, exist_ok=True)

    template_filepath = workdir/template_filename
    with open(template_filepath.absolute(), "r", encoding="utf-8") as f:
        template = f.read()
        
    run_script = template.replace("@INSTANCE@", instance_name)
    run_script = run_script.replace("@TIME_LIMIT@", str(time_limit))
    run_script = run_script.replace("@MODEL@", model_filename)
    run_script_filename = f"run_{instance_name}.run"
    run_script_filepath = workdir/run_script_filename
    with open(run_script_filepath.absolute(), "w+", encoding="utf-8") as f:
        f.write(run_script)
    cmd_args = ["ampl", run_script_filename]
    try:
        result = subprocess.run(cmd_args, 
                                cwd=workdir.absolute(),
                                capture_output=True, 
                                text=True, 
                                check=True)
        # print(result.stdout)
    except subprocess.CalledProcessError as e:
        print("Command failed!")
        print("Return code:", e.returncode)
        print("Output:", e.output)
        print("Error output:", e.stderr)
        return "",""
    except FileNotFoundError:
        print("Command not found:", cmd_args[0])
        return "",""
    finally:
        if os.path.exists(run_script_filepath.absolute()):
            os.remove(run_script_filepath.absolute())
        # print(f"File {run_script_filename} has been removed.")
    
    output_txt: str
    output_filepath = output_dir/f"{instance_name}.out"
    with open(output_filepath.absolute(), "r", encoding="utf-8") as f:
        output_txt = f.read()
    return instance_name, output_txt

In [10]:
time_limit = 14400
instance_names = [
    "A-n6-k2-m0-b2ampl_.txt",
    "A-n6-k2-m1-b2ampl_.txt",
    "A-n6-k2-m2-b2ampl_.txt",
    "A-n6-k2-m3-b2ampl_.txt",
    "A-n7-k2-m0-b2ampl_.txt",
    "A-n7-k2-m1-b2ampl_.txt",
    "A-n7-k2-m2-b2ampl_.txt",
    "A-n7-k2-m3-b2ampl_.txt",
    "A-n8-k3-m0-b3ampl_.txt",
    "A-n8-k3-m1-b3ampl_.txt",
    "A-n8-k3-m2-b3ampl_.txt",
    "A-n8-k3-m3-b3ampl_.txt",
    "A-n9-k3-m0-b3ampl_.txt",
    "A-n9-k3-m1-b3ampl_.txt",
    "A-n9-k3-m2-b3ampl_.txt",
    "A-n9-k3-m3-b3ampl_.txt",
    "A-n10-k3-m0-b3ampl_.txt",
    "A-n10-k3-m1-b3ampl_.txt",
    "A-n10-k3-m2-b3ampl_.txt",
    "A-n10-k3-m3-b3ampl_.txt",
    "A-n11-k4-m0-b4ampl_.txt",
    "A-n11-k4-m1-b4ampl_.txt",
    "A-n11-k4-m2-b4ampl_.txt",
    "A-n11-k4-m3-b4ampl_.txt",
    "A-n12-k4-m0-b4ampl_.txt",
    "A-n12-k4-m1-b4ampl_.txt",
    "A-n12-k4-m2-b4ampl_.txt",
    "A-n12-k4-m3-b4ampl_.txt",
    "A-n13-k4-m0-b4ampl_.txt",
    "A-n13-k4-m1-b4ampl_.txt",
    "A-n13-k4-m2-b4ampl_.txt",
    "A-n13-k4-m3-b4ampl_.txt",
    "A-n14-k4-m0-b4ampl_.txt",
    "A-n14-k4-m1-b4ampl_.txt",
    "A-n14-k4-m2-b4ampl_.txt",
    "A-n14-k4-m3-b4ampl_.txt",
    "A-n15-k4-m0-b4ampl_.txt",
    "A-n15-k4-m1-b4ampl_.txt",
    "A-n15-k4-m2-b4ampl_.txt",
    "A-n15-k4-m3-b4ampl_.txt",
    "A-n16-k4-m0-b4ampl_.txt",
    "A-n16-k4-m1-b4ampl_.txt",
    "A-n16-k4-m2-b4ampl_.txt",
    "A-n16-k4-m3-b4ampl_.txt",
    "A-n17-k4-m0-b4ampl_.txt",
    "A-n17-k4-m1-b4ampl_.txt",
    "A-n17-k4-m2-b4ampl_.txt",
    "A-n17-k4-m3-b4ampl_.txt",
    "A-n18-k4-m0-b4ampl_.txt",
    "A-n18-k4-m1-b4ampl_.txt",
    "A-n18-k4-m2-b4ampl_.txt",
    "A-n18-k4-m3-b4ampl_.txt",
    "A-n19-k4-m0-b4ampl_.txt",
    "A-n19-k4-m1-b4ampl_.txt",
    "A-n19-k4-m2-b4ampl_.txt",
    "A-n19-k4-m3-b4ampl_.txt",
    "A-n20-k4-m0-b4ampl_.txt",
    "A-n20-k4-m1-b4ampl_.txt",
    "A-n20-k4-m2-b4ampl_.txt",
    "A-n20-k4-m3-b4ampl_.txt",
    "A-n21-k4-m0-b4ampl_.txt",
    "A-n21-k4-m1-b4ampl_.txt",
    "A-n21-k4-m2-b4ampl_.txt",
    "A-n21-k4-m3-b4ampl_.txt",
    "A-n22-k4-m0-b4ampl_.txt",
    "A-n22-k4-m1-b4ampl_.txt",
    "A-n22-k4-m2-b4ampl_.txt",
    "A-n22-k4-m3-b4ampl_.txt",
    "A-n23-k4-m0-b4ampl_.txt",
    "A-n23-k4-m1-b4ampl_.txt",
    "A-n23-k4-m2-b4ampl_.txt",
    "A-n23-k4-m3-b4ampl_.txt",
    "A-n24-k4-m0-b4ampl_.txt",
    "A-n24-k4-m1-b4ampl_.txt",
    "A-n24-k4-m2-b4ampl_.txt",
    "A-n24-k4-m3-b4ampl_.txt",
    "A-n25-k4-m0-b4ampl_.txt",
    "A-n25-k4-m1-b4ampl_.txt",
    "A-n25-k4-m2-b4ampl_.txt",
    "A-n25-k4-m3-b4ampl_.txt",
    "A-n26-k4-m0-b4ampl_.txt",
    "A-n26-k4-m1-b4ampl_.txt",
    "A-n26-k4-m2-b4ampl_.txt",
    "A-n26-k4-m3-b4ampl_.txt",
    "A-n32-k5-m0-b5ampl_.txt",
    "A-n32-k5-m1-b5ampl_.txt",
    "A-n32-k5-m2-b5ampl_.txt",
    "A-n32-k5-m3-b5ampl_.txt",
    "A-n33-k5-m0-b5ampl_.txt",
    "A-n33-k5-m1-b5ampl_.txt",
    "A-n33-k5-m2-b5ampl_.txt",
    "A-n33-k5-m3-b5ampl_.txt",
    "A-n33-k6-m0-b6ampl_.txt",
    "A-n33-k6-m1-b6ampl_.txt",
    "A-n33-k6-m2-b6ampl_.txt",
    "A-n33-k6-m3-b6ampl_.txt",
    "A-n34-k5-m0-b5ampl_.txt",
    "A-n34-k5-m1-b5ampl_.txt",
    "A-n34-k5-m2-b5ampl_.txt",
    "A-n34-k5-m3-b5ampl_.txt",
    "A-n36-k5-m0-b5ampl_.txt",
    "A-n36-k5-m1-b5ampl_.txt",
    "A-n36-k5-m2-b5ampl_.txt",
    "A-n36-k5-m3-b5ampl_.txt",
    "A-n37-k5-m0-b5ampl_.txt",
    "A-n37-k5-m1-b5ampl_.txt",
    "A-n37-k5-m2-b5ampl_.txt",
    "A-n37-k5-m3-b5ampl_.txt",
    "A-n37-k6-m0-b6ampl_.txt",
    "A-n37-k6-m1-b6ampl_.txt",
    "A-n37-k6-m2-b6ampl_.txt",
    "A-n37-k6-m3-b6ampl_.txt",
    "A-n38-k5-m0-b5ampl_.txt",
    "A-n38-k5-m1-b5ampl_.txt",
    "A-n38-k5-m2-b5ampl_.txt",
    "A-n38-k5-m3-b5ampl_.txt",
    "A-n39-k5-m0-b5ampl_.txt",
    "A-n39-k5-m1-b5ampl_.txt",
    "A-n39-k5-m2-b5ampl_.txt",
    "A-n39-k5-m3-b5ampl_.txt",
    "A-n39-k6-m0-b6ampl_.txt",
    "A-n39-k6-m1-b6ampl_.txt",
    "A-n39-k6-m2-b6ampl_.txt",
    "A-n39-k6-m3-b6ampl_.txt",
    "A-n44-k6-m0-b6ampl_.txt",
    "A-n44-k6-m1-b6ampl_.txt",
    "A-n44-k6-m2-b6ampl_.txt",
    "A-n44-k6-m3-b6ampl_.txt",
    "A-n45-k6-m0-b6ampl_.txt",
    "A-n45-k6-m1-b6ampl_.txt",
    "A-n45-k6-m2-b6ampl_.txt",
    "A-n45-k6-m3-b6ampl_.txt",
    "A-n45-k7-m0-b7ampl_.txt",
    "A-n45-k7-m1-b7ampl_.txt",
    "A-n45-k7-m2-b7ampl_.txt",
    "A-n45-k7-m3-b7ampl_.txt",
    "A-n46-k7-m0-b7ampl_.txt",
    "A-n46-k7-m1-b7ampl_.txt",
    "A-n46-k7-m2-b7ampl_.txt",
    "A-n46-k7-m3-b7ampl_.txt",
    "A-n48-k7-m0-b7ampl_.txt",
    "A-n48-k7-m1-b7ampl_.txt",
    "A-n48-k7-m2-b7ampl_.txt",
    "A-n48-k7-m3-b7ampl_.txt",
    "A-n53-k7-m0-b7ampl_.txt",
    "A-n53-k7-m1-b7ampl_.txt",
    "A-n53-k7-m2-b7ampl_.txt",
    "A-n53-k7-m3-b7ampl_.txt",
    "A-n54-k7-m0-b7ampl_.txt",
    "A-n54-k7-m1-b7ampl_.txt",
    "A-n54-k7-m2-b7ampl_.txt",
    "A-n54-k7-m3-b7ampl_.txt",
    "A-n55-k9-m0-b9ampl_.txt",
    "A-n55-k9-m1-b9ampl_.txt",
    "A-n55-k9-m2-b9ampl_.txt",
    "A-n55-k9-m3-b9ampl_.txt",
    "A-n60-k9-m0-b9ampl_.txt",
    "A-n60-k9-m1-b9ampl_.txt",
    "A-n60-k9-m2-b9ampl_.txt",
    "A-n60-k9-m3-b9ampl_.txt",
    "A-n61-k9-m0-b9ampl_.txt",
    "A-n61-k9-m1-b9ampl_.txt",
    "A-n61-k9-m2-b9ampl_.txt",
    "A-n61-k9-m3-b9ampl_.txt",
    "A-n62-k8-m0-b8ampl_.txt",
    "A-n62-k8-m1-b8ampl_.txt",
    "A-n62-k8-m2-b8ampl_.txt",
    "A-n62-k8-m3-b8ampl_.txt",
    "A-n63-k10-m0-b10ampl_.txt",
    "A-n63-k10-m1-b10ampl_.txt",
    "A-n63-k10-m2-b10ampl_.txt",
    "A-n63-k10-m3-b10ampl_.txt",
    "A-n63-k9-m0-b9ampl_.txt",
    "A-n63-k9-m1-b9ampl_.txt",
    "A-n63-k9-m2-b9ampl_.txt",
    "A-n63-k9-m3-b9ampl_.txt",
    "A-n64-k9-m0-b9ampl_.txt",
    "A-n64-k9-m1-b9ampl_.txt",
    "A-n64-k9-m2-b9ampl_.txt",
    "A-n64-k9-m3-b9ampl_.txt",
    "A-n65-k9-m0-b9ampl_.txt",
    "A-n65-k9-m1-b9ampl_.txt",
    "A-n65-k9-m2-b9ampl_.txt",
    "A-n65-k9-m3-b9ampl_.txt",
    "A-n69-k9-m0-b9ampl_.txt",
    "A-n69-k9-m1-b9ampl_.txt",
    "A-n69-k9-m2-b9ampl_.txt",
    "A-n69-k9-m3-b9ampl_.txt",
    "A-n80-k10-m0-b10ampl_.txt",
    "A-n80-k10-m1-b10ampl_.txt",
    "A-n80-k10-m2-b10ampl_.txt",
    "A-n80-k10-m3-b10ampl_.txt",
]

In [ ]:
futures = [call_ampl.remote(instance_name, time_limit) for instance_name in instance_names]
output_dir = pathlib.Path("ampl_outputs")
output_dir.mkdir(parents=True, exist_ok=True)
while futures:
    ready_ids, futures = ray.wait(futures, num_returns=1)
    for obj_id in ready_ids:
        result = ray.get(obj_id)
        instance_name, output_txt = result
        output_filepath = output_dir/f"{instance_name}.out"
        if output_filepath.exists():
            print(f"[Skip] {instance_name} already written.")
            continue
        output_filepath.write_text(output_txt, encoding="utf-8")
        print(f"[Done] {instance_name} → {output_filepath}")